In [1]:
import pandas as pd
import torch
import json
import numpy as np
from torch.utils.data import random_split, DataLoader
from neural_test import train_model, test_model, CustomDataset, make_and_eval
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt

In [2]:
class DynamicNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_hidden_layers):
        super(DynamicNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_hidden_layers = num_hidden_layers

        # Create the first layer
        self.layers = [torch.nn.Linear(input_size, hidden_size), torch.nn.ReLU()]

        # Create the hidden layers
        for _ in range(num_hidden_layers):
            self.layers.append(torch.nn.Linear(hidden_size, hidden_size))
            self.layers.append(torch.nn.ReLU())

        # Create the output layer
        self.layers.append(torch.nn.Linear(hidden_size, output_size))

        # Combine all layers
        self.model = torch.nn.Sequential(*self.layers)

    def forward(self, x):
        out = self.model(x)
        if not self.training:
            out = torch.clamp(out, min=1, max=5)
        return out

In [3]:
device = "cpu"
# Load JSON data
with open('topic-sentiment-total.json') as file:
    json_data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame.from_dict(json_data, orient='index')

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert DataFrame to numpy array
data = df.to_numpy()
df.head()

,pos_food,pos_service,pos_location,pos_clean,pos_price,neg_food,neg_service,neg_location,neg_clean,neg_price,rating
0,20,7,2,1,0,7,1,0,0,1,4.3
1,119,37,38,3,11,31,2,0,0,2,4.6
2,228,85,59,4,53,100,42,19,1,23,4.2
3,22,10,2,2,2,4,4,0,1,2,4.2
4,60,32,37,0,7,32,12,8,0,14,4.1


In [4]:
input_dim = 10
hidden_dim = 40
num_hidden_layers = 4
output_dim = 1  
learningRate = .005
epochs = 50

lambda1 = 0.0000 # l1 regularization parameter (sum of weights)
lambda2 = 0.0000 # l2 regularization parameter (sum of square of weights)

model = DynamicNN(input_dim, hidden_dim, output_dim, num_hidden_layers).to(device)
# Split the dataset
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(42))

trained_model, _ = make_and_eval(model, np.array(train_data), np.array(test_data), learningRate, epochs, lambda1, lambda2)

Mean Squared Error: 0.05780668929219246
Root Mean Squared Error: 0.24043022096157074
R-squared: 0.6724168697245924


In [5]:
torch.save(model.state_dict(), "nn4_40.pt")

In [6]:
model = DynamicNN(input_dim, hidden_dim, output_dim, num_hidden_layers).to(device)
k_folds = 5
# Create KFold object
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

models: list[torch.nn.Module] = []

for fold, (train_ids, test_ids) in enumerate(kf.split(data)):
    print(f"Fold {fold + 1}/{k_folds}")

    trained_model, _ = make_and_eval(model, data[train_ids], data[test_ids], learningRate, epochs, lambda1, lambda2)

    models.append(trained_model)


Fold 1/5


: 

In [ ]:

class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        if not self.training:
            out = torch.clamp(out, min=1, max=5)
        return out

In [ ]:
# Load JSON data
with open('restaurant-topic-sentiment.json') as file:
    json_data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame.from_dict(json_data, orient='index')

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert DataFrame to numpy array
data = df.to_numpy()
df.head()

,pos_food,pos_service,pos_location,pos_clean,pos_price,neg_food,neg_service,neg_location,neg_clean,neg_price,rating
0,1.538462,0.538462,0.153846,0.076923,0.000000,0.538462,0.076923,0.000000,0.000000,0.076923,4.3
1,1.525641,0.474359,0.487179,0.038462,0.141026,0.397436,0.025641,0.000000,0.000000,0.025641,4.6
2,0.850746,0.317164,0.220149,0.014925,0.197761,0.373134,0.156716,0.070896,0.003731,0.085821,4.2
3,0.431373,0.196078,0.039216,0.039216,0.039216,0.078431,0.078431,0.000000,0.019608,0.039216,4.2
4,1.000000,0.533333,0.616667,0.000000,0.116667,0.533333,0.200000,0.133333,0.000000,0.233333,4.1


In [ ]:
input_dim = 10
output_dim = 1
learningRate = .015
epochs = 100

lambda1 = 0.0005 # l1 regularization parameter (sum of weights)
lambda2 = 0.0005 # l2 regularization parameter (sum of square of weights)

model = linearRegression(input_dim, output_dim).to(device)

k_folds = 5
# Create KFold object
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

models: list[torch.nn.Module] = []
all_mse = []
all_rmse = []
all_r2 = []

for fold, (train_ids, test_ids) in enumerate(kf.split(data)):
    print(f"Fold {fold + 1}/{k_folds}")

    trained_model, evals = make_and_eval(model, data[train_ids], data[test_ids], learningRate, epochs, lambda1, lambda2)
    
    models.append(trained_model)
    all_mse.append(evals[1])
    all_rmse.append(evals[2])
    all_r2.append(evals[3])

# Calculate and print the average of the metrics
print(f'Average Mean Squared Error: {np.mean(all_mse)}')
print(f'Average Root Mean Squared Error: {np.mean(all_rmse)}')
print(f'Average R-squared: {np.mean(all_r2)}')

Fold 1/5
Mean Squared Error: 0.052503395825624466
Root Mean Squared Error: 0.22913619875907898
R-squared: 0.6994251172259214
Fold 2/5
Mean Squared Error: 0.05128667131066322
Root Mean Squared Error: 0.2264656126499176
R-squared: 0.6754854034349593
Fold 3/5
Mean Squared Error: 0.0627734363079071
Root Mean Squared Error: 0.25054627656936646
R-squared: 0.643272579849389
Fold 4/5
Mean Squared Error: 0.05888679251074791
Root Mean Squared Error: 0.24266600608825684
R-squared: 0.7019252833137355
Fold 5/5
Mean Squared Error: 0.06590893119573593
Root Mean Squared Error: 0.25672733783721924
R-squared: 0.6287286201172206
